# 🚚 Smart Logistics: IoT Data Simulation

This notebook simulates **IoT-generated data** for a **smart logistics tracking system**, inspired by how platforms like **Shopee**, **Lazada**, and **Lalamove** monitor their deliveries in the Philippines.

The simulation captures real-world shipment behavior by generating random, yet realistic, sensor data and delivery event details. The goal is to understand how logistics systems track environmental conditions and shipment events to maintain product quality and delivery performance.

---

## 📦 Objectives

✅ Simulate realistic shipment tracking and delivery records using Python.  
✅ Understand how **IoT sensors** monitor environmental data like temperature and humidity.  
✅ Include business-relevant details such as delay detection, alert flags, and shipment statuses.  
✅ Generate datasets for use in **visualization**, **analytics**, and potential smart contract or blockchain integrations.

In [4]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import os
import json

We’ll store our simulated data in a subfolder called `data/`.

In [5]:
os.makedirs("../data", exist_ok=True)

Each shipment record includes:

- **Timestamps**:
  - `timestamp`: when the data was recorded
  - `pickup_datetime`: when the item was picked up
  - `estimated_arrival`: when the item was expected to arrive
  - `actual_arrival`: when the item actually arrived

- **Identifiers**:
  - `shipment_id`: a unique tracking ID
  - `order_id`: the customer's original order number

- **Logistics Routing**:
  - `origin_hub`: where the item was dispatched from (e.g., Cebu Hub, QC Hub)
  - `destination_city`: delivery city (e.g., Taguig, Baguio)

- **Status & Delivery**:
  - `status`: current status of the shipment (e.g., Delivered, Delayed)
  - `logistics_delay`: flag if actual arrival is later than estimated
  - `logistics_delay_reason`: reason for delay (if any)
  - `customer_notified`: whether the customer has been updated

- **Environmental Monitoring (IoT Simulation)**:
  - `temperature_c`: recorded temperature in °C (cold-chain packages)
  - `humidity_percent`: humidity level in %
  - `temperature_alert`: flagged if outside acceptable range (2.5–7.5°C)
  - `humidity_alert`: flagged if outside acceptable range (55–75%)

- **Package Details**:
  - `package_type`: type of item (e.g., Pouch, Small Box, Frozen Pack)
  - `delivery_rating`: customer rating after delivery 

In [ ]:
num_records = 100
data = []

for _ in range(num_records):
    # Generate timestamps
    timestamp = datetime.now() - timedelta(minutes=np.random.randint(0, 1440))
    pickup_datetime = timestamp - timedelta(hours=np.random.randint(1, 6))
    estimated_arrival = timestamp + timedelta(hours=np.random.randint(24, 49))
    actual_arrival = estimated_arrival + timedelta(minutes=np.random.randint(-60, 120))

    # Simulated IoT sensor readings
    temperature_c = round(np.random.uniform(2.0, 8.0), 2)
    humidity_percent = round(np.random.uniform(50.0, 80.0), 2)

    # Alert logic using native bool
    temperature_alert = bool(temperature_c < 2.5 or temperature_c > 7.5)
    humidity_alert = bool(humidity_percent < 55 or humidity_percent > 75)

    # Delay detection
    logistics_delay = actual_arrival > estimated_arrival
    logistics_delay_reason = (
        np.random.choice(["Heavy Traffic", "Flooded Roads", "Mechanical Issue", "Checkpoint Delay"])
        if logistics_delay else "None"
    )

    record = {
        "timestamp": timestamp.strftime("%Y-%m-%d %H:%M:%S"),
        "pickup_datetime": pickup_datetime.strftime("%Y-%m-%d %H:%M:%S"),
        "estimated_arrival": estimated_arrival.strftime("%Y-%m-%d %H:%M:%S"),
        "actual_arrival": actual_arrival.strftime("%Y-%m-%d %H:%M:%S"),
        "shipment_id": f"SHIP{np.random.randint(1000, 9999)}",
        "order_id": f"ORD{np.random.randint(100000, 999999)}",
        "origin_hub": np.random.choice(["Cebu Hub", "QC Hub", "Laguna Warehouse", "Davao Center"]),
        "destination_city": np.random.choice(["Taguig", "Cebu City", "Baguio", "Davao", "Naga", "Iloilo"]),
        "status": np.random.choice(["In Transit", "Delivered", "Delayed", "Out for Delivery"]),
        "temperature_c": temperature_c,
        "humidity_percent": humidity_percent,
        "temperature_alert": temperature_alert,
        "humidity_alert": humidity_alert,
        "package_type": np.random.choice(["Pouch", "Small Box", "Medium Box", "Large Box", "Envelope", "Frozen Pack"]),
        "logistics_delay": bool(logistics_delay),
        "logistics_delay_reason": logistics_delay_reason,
        "delivery_rating": random.choice([None, 3, 4, 5]),
        "customer_notified": bool(np.random.choice([True, False]))
    }

    data.append(record)

# Convert to DataFrame
df = pd.DataFrame(data)

We’ll export the dataset in two formats:
- CSV for spreadsheet tools
- JSON for API and web usage

In [10]:
# Save to CSV and JSON
df.to_csv("../data/logistics_data.csv", index=False)

import json

with open("../data/logistics_data.json", "w") as json_file:
    json.dump(data, json_file, indent=4)

In [11]:
df.head()

,timestamp,pickup_datetime,estimated_arrival,actual_arrival,shipment_id,order_id,origin_hub,destination_city,status,temperature_c,humidity_percent,temperature_alert,humidity_alert,package_type,logistics_delay,logistics_delay_reason,delivery_rating,customer_notified
0,2025-05-18 15:46:54,2025-05-18 11:46:54,2025-05-20 14:46:54,2025-05-20 15:28:54,SHIP7734,ORD171327,Davao Center,Naga,Delivered,2.23,53.55,True,True,Medium Box,True,Flooded Roads,5.0,False
1,2025-05-18 09:41:54,2025-05-18 06:41:54,2025-05-19 09:41:54,2025-05-19 10:57:54,SHIP4309,ORD162097,QC Hub,Taguig,Delivered,6.28,57.19,False,False,Envelope,True,Flooded Roads,5.0,False
2,2025-05-18 13:45:54,2025-05-18 10:45:54,2025-05-19 20:45:54,2025-05-19 20:15:54,SHIP1802,ORD279145,Davao Center,Baguio,Out for Delivery,6.14,71.31,False,False,Envelope,False,None,3.0,False
3,2025-05-18 05:57:54,2025-05-18 01:57:54,2025-05-19 06:57:54,2025-05-19 07:38:54,SHIP9967,ORD542396,Cebu Hub,Iloilo,In Transit,3.09,70.90,False,False,Pouch,True,Heavy Traffic,4.0,False
4,2025-05-18 01:22:54,2025-05-17 22:22:54,2025-05-19 16:22:54,2025-05-19 18:07:54,SHIP3088,ORD752230,QC Hub,Baguio,Out for Delivery,4.69,61.32,False,False,Medium Box,True,Heavy Traffic,3.0,False
